In [100]:
import boto3
import pandas as pd
from datetime import datetime
import re
import numpy as np
import time


# Load and clean all_tournaments
all_tournaments = pd.read_csv('data/all_tournaments2020.csv')

year = [match.replace(' ','').split('-')[0].split('.')[0] for match in all_tournaments.date]
duree_tournois = [(datetime.strptime(match.replace(' ','').split('-')[1] , '%Y.%m.%d').date() - datetime.strptime(match.replace(' ','').split('-')[0] , '%Y.%m.%d').date()).days for match in all_tournaments.date]
year = [match.replace(' ','').split('-')[0].split('.')[0] for match in all_tournaments.date]
all_tournaments.insert(2, "year", year, True) 
all_tournaments.insert(3, "duree", duree_tournois, True)
all_tournaments.winnerRate = all_tournaments.winnerRate.str.replace(r'[^0-9]+', '')
all_tournaments.looserRate = all_tournaments.looserRate.str.replace(r'[^0-9]+', '')

# Load and add surface all_players

all_players = pd.read_csv('data/all_players2020.csv')

surface = [url.split('=')[2] for url in all_players.url]
all_players.insert(3, "surface", surface, True)

In [101]:
def getPlayStats(player, year, surface):
    
    if "Hard" in surface:
        surface = "hard"
    if "Clay" in surface:
        surface = "clay"
    if "Grass" in surface or "Carpet" in surface:
        surface = "grass"
                
    df = all_players[(all_players.year==year) & (all_players.player==player) & (all_players.surface==surface)]
        
    if df.empty:
        columns = ['player', 'aces', 'doubleFautes', 'firstServe', 'firstServePointsWon',
       'secondServePointsWon', 'breakPointsFaced', 'breakPointsSaved',
       'serviceGamePlayed', 'serviceGamesWon', 'totalServicePointsWon',
       'firstServeReturnPointsWon', 'secondServeReturnPointsWon',
       'breakPointsOpportunities', 'breakPointsConverted', 'returnGamesPlayed',
       'returnGamesWon', 'returnPointsWon', 'totalPointsWon', 'url', 'year',
       'weight', 'height', 'age', 'startCareer', 'plays', 'ranking', 'points',
       'tournCell', 'pointsDropping', 'nextBest']
        a = np.empty(len(columns))
        a[:] = np.nan
        newDF = pd.DataFrame([a], columns=columns)
        return newDF
    else:
        return df.iloc[-1]

label = []

for (winner, looser)  in zip(all_tournaments.winnerName, all_tournaments.looserName):
    if min(winner, looser) == winner:
        label.append("joueur0")
    else:
        label.append("joueur1")
        
all_tournaments['label'] = label

In [102]:
df_0 = pd.DataFrame([], columns=['player', 'aces', 'doubleFautes', 'firstServe', 'firstServePointsWon',
       'secondServePointsWon', 'breakPointsFaced', 'breakPointsSaved',
       'serviceGamePlayed', 'serviceGamesWon', 'totalServicePointsWon',
       'firstServeReturnPointsWon', 'secondServeReturnPointsWon',
       'breakPointsOpportunities', 'breakPointsConverted', 'returnGamesPlayed',
       'returnGamesWon', 'returnPointsWon', 'totalPointsWon', 'url', 'year',
       'weight', 'height', 'age', 'startCareer', 'plays', 'ranking', 'points',
       'tournCell', 'pointsDropping', 'nextBest'])

df_1 = pd.DataFrame([], columns=['player', 'aces', 'doubleFautes', 'firstServe', 'firstServePointsWon',
       'secondServePointsWon', 'breakPointsFaced', 'breakPointsSaved',
       'serviceGamePlayed', 'serviceGamesWon', 'totalServicePointsWon',
       'firstServeReturnPointsWon', 'secondServeReturnPointsWon',
       'breakPointsOpportunities', 'breakPointsConverted', 'returnGamesPlayed',
       'returnGamesWon', 'returnPointsWon', 'totalPointsWon', 'url', 'year',
       'weight', 'height', 'age', 'startCareer', 'plays', 'ranking', 'points',
       'tournCell', 'pointsDropping', 'nextBest'])

total_size = len(all_tournaments)
start_time = time.time()

for i, (winnerName, looserName, year, surface) in enumerate(zip(all_tournaments.winnerName, all_tournaments.looserName, all_tournaments.year, all_tournaments.surfaceInout)):

    if min(winnerName, looserName) == winnerName:
        df_0 = df_0.append(getPlayStats(winnerName, int(year), surface), ignore_index=True)
        df_1 = df_1.append(getPlayStats(looserName, int(year), surface), ignore_index=True)
    else:
        df_0 = df_0.append(getPlayStats(looserName, int(year), surface), ignore_index=True)
        df_1 = df_1.append(getPlayStats(winnerName, int(year), surface), ignore_index=True)
        


In [103]:
df_1.columns = ['aces1', 'age1', 'breakPointsConverted1', 'breakPointsFaced1',
       'breakPointsOpportunities1', 'breakPointsSaved1', 'doubleFautes1',
       'firstServe1', 'firstServePointsWon1', 'firstServeReturnPointsWon1',
       'height1', 'nextBest1', 'player1', 'plays1', 'points1', 'pointsDropping1',
       'ranking1', 'returnGamesPlayed1', 'returnGamesWon1', 'returnPointsWon1',
       'secondServePointsWon1', 'secondServeReturnPointsWon1',
       'serviceGamePlayed1', 'serviceGamesWon1', 'startCareer1', 'surface1',
       'totalPointsWon1', 'totalServicePointsWon1', 'tournCell1', 'url1', 'weight1',
       'year1']

df_0.columns = ['aces0', 'age0', 'breakPointsConverted0', 'breakPointsFaced0',
       'breakPointsOpportunities0', 'breakPointsSaved0', 'doubleFautes0',
       'firstServe0', 'firstServePointsWon0', 'firstServeReturnPointsWon0',
       'height0', 'nextBest0', 'player0', 'plays0', 'points0', 'pointsDropping0',
       'ranking0', 'returnGamesPlayed0', 'returnGamesWon0', 'returnPointsWon0',
       'secondServePointsWon0', 'secondServeReturnPointsWon0',
       'serviceGamePlayed0', 'serviceGamesWon0', 'startCareer0', 'surface0',
       'totalPointsWon0', 'totalServicePointsWon0', 'tournCell0', 'url0', 'weight0',
       'year0']

all_tournaments_with_stats = pd.concat([all_tournaments.reset_index(), df_0.reset_index(), df_1.reset_index()], axis=1)

In [104]:
all_tournaments_with_stats.breakPointsConverted0 = pd.to_numeric(all_tournaments_with_stats.breakPointsConverted0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.breakPointsSaved0 = pd.to_numeric(all_tournaments_with_stats.breakPointsSaved0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServe0 = pd.to_numeric(all_tournaments_with_stats.firstServe0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServePointsWon0 = pd.to_numeric(all_tournaments_with_stats.firstServePointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServeReturnPointsWon0 = pd.to_numeric(all_tournaments_with_stats.firstServeReturnPointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.serviceGamesWon0 = pd.to_numeric(all_tournaments_with_stats.serviceGamesWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.totalPointsWon0 = pd.to_numeric(all_tournaments_with_stats.totalPointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.totalServicePointsWon0 = pd.to_numeric(all_tournaments_with_stats.totalServicePointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.height0 = pd.to_numeric(all_tournaments_with_stats.height0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.returnGamesWon0 = pd.to_numeric(all_tournaments_with_stats.returnGamesWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.returnPointsWon0 = pd.to_numeric(all_tournaments_with_stats.returnPointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.secondServePointsWon0 = pd.to_numeric(all_tournaments_with_stats.secondServePointsWon0.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.secondServeReturnPointsWon0 = pd.to_numeric(all_tournaments_with_stats.secondServeReturnPointsWon0.str.replace(r'[^0-9]+', ''))

date = []
for y, s in zip(all_tournaments_with_stats.year, all_tournaments_with_stats.age0):
    try:
        date.append(int(y) - datetime.strptime(s[s.find("(")+1:s.find(")")], '%Y.%m.%d').date().year)
    except:
        date.append("NaN")
        
all_tournaments_with_stats.age0 = date


all_tournaments_with_stats.breakPointsConverted1 = pd.to_numeric(all_tournaments_with_stats.breakPointsConverted1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.breakPointsSaved1 = pd.to_numeric(all_tournaments_with_stats.breakPointsSaved1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServe1 = pd.to_numeric(all_tournaments_with_stats.firstServe1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServePointsWon1 = pd.to_numeric(all_tournaments_with_stats.firstServePointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.firstServeReturnPointsWon1 = pd.to_numeric(all_tournaments_with_stats.firstServeReturnPointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.serviceGamesWon1 = pd.to_numeric(all_tournaments_with_stats.serviceGamesWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.totalPointsWon1= pd.to_numeric(all_tournaments_with_stats.totalPointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.totalServicePointsWon1 = pd.to_numeric(all_tournaments_with_stats.totalServicePointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.height1 = pd.to_numeric(all_tournaments_with_stats.height1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.returnGamesWon1 = pd.to_numeric(all_tournaments_with_stats.returnGamesWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.returnPointsWon1 = pd.to_numeric(all_tournaments_with_stats.returnPointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.secondServePointsWon1 = pd.to_numeric(all_tournaments_with_stats.secondServePointsWon1.str.replace(r'[^0-9]+', ''))
all_tournaments_with_stats.secondServeReturnPointsWon1 = pd.to_numeric(all_tournaments_with_stats.secondServeReturnPointsWon1.str.replace(r'[^0-9]+', ''))

date = []
for y, s in zip(all_tournaments_with_stats.year, all_tournaments_with_stats.age1):
    try:
        date.append(int(y) - datetime.strptime(s[s.find("(")+1:s.find(")")], '%Y.%m.%d').date().year)
    except:
        date.append("NaN")
        
all_tournaments_with_stats.age1 = date

In [108]:
all_tournaments_with_stats = pd.concat([all_tournaments.reset_index(drop=True), df_0.reset_index(drop=True), df_1.reset_index(drop=True)], axis=1, sort=False) 
all_tournaments_with_stats.to_csv('data/all_tournaments_with_stats2020.csv', index=False)


In [107]:
all_tournaments_with_stats

,index,name,phase,year,duree,surfaceInout,typeTournament,date,winnerRate,looserRate,...,serviceGamePlayed1,serviceGamesWon1,startCareer1,surface1,totalPointsWon1,totalServicePointsWon1,tournCell1,url1,weight1,year1
0,0,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,15,93.0,2017,hard,52.0,70.0,25,https://www.atptour.com/en/players/alexei-popy...,172,2020
1,1,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,114,77.0,2009,hard,49.0,62.0,25,https://www.atptour.com/en/players/henri-laaks...,172,2020
2,2,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,17,59.0,2016,hard,43.0,56.0,30,https://www.atptour.com/en/players/elliot-benc...,194,2020
3,3,montpellier,Round of 32,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,[],NaN,2012,hard,NaN,NaN,26,https://www.atptour.com/en/players/quentin-hal...,172,2020
4,4,montpellier,Qualification,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,0,0,...,8,13.0,\r\n,hard,37.0,35.0,28,https://www.atptour.com/en/players/hiroki-mori...,143,2020
5,5,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,42,74.0,2009,hard,53.0,61.0,27,https://www.atptour.com/en/players/david-goffi...,154,2020
6,6,montpellier,Qualification,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,0,0,...,38,76.0,2005,hard,47.0,63.0,19,https://www.atptour.com/en/players/lukas-lacko...,192,2020
7,7,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,0,...,0,0.0,2018,hard,0.0,0.0,23,https://www.atptour.com/en/players/emil-ruusuv...,174,2020
8,8,montpellier,Round of 32,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,0,0,...,49,76.0,2005,hard,50.0,61.0,18,https://www.atptour.com/en/players/matthias-ba...,190,2020
9,9,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,0,0,...,11,91.0,2000,hard,48.0,69.0,10,https://www.atptour.com/en/players/nicolas-mah...,181,2020
